In [ ]:
pip install transformers

In [92]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import regex as re

In [93]:
data = pd.read_csv('mbti_1.csv')
data

,type,posts
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...
1,ENTP,'I'm finding the lack of me in these posts ver...
2,INTP,'Good one _____ https://www.youtube.com/wat...
3,INTJ,"'Dear INTP, I enjoyed our conversation the o..."
4,ENTJ,'You're fired.|||That's another silly misconce...
...,...,...
8670,ISFP,'https://www.youtube.com/watch?v=t8edHB_h908||...
8671,ENFP,'So...if this thread already exists someplace ...
8672,INTP,'So many questions when i do these things. I ...
8673,INFP,'I am very conflicted right now when it comes ...


In [94]:
df=data[data.index<=500]

In [95]:
data=df

In [96]:
def clean_text(data):
    data_length=[]
    cleaned_text=[]
    for sentence in data.posts:
        sentence=sentence.lower()
        
        #removing links from text data
        sentence=re.sub('https?://[^\s<>"]+|www\.[^\s<>"]+',' ',sentence)
    
        #removing other symbols
        sentence=re.sub('[^0-9a-z]',' ',sentence)
    
        
        data_length.append(len(sentence.split()))
        cleaned_text.append(sentence)
    return cleaned_text
data.posts = clean_text(data)
data

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,type,posts
0,INFJ,and intj moments sportscenter not top t...
1,ENTP,i m finding the lack of me in these posts ver...
2,INTP,good one course to which i say i ...
3,INTJ,dear intp i enjoyed our conversation the o...
4,ENTJ,you re fired that s another silly misconce...
...,...,...
496,INFJ,answer the subject of the thread you won ...
497,ENTP,well when you figure out where to begin plea...
498,INFJ,dude that sounds fantastic if you ever put a...
499,INTJ,just picked this post to reply to but ive re...


In [97]:
data.type = data.apply(lambda x:1 if 'I' in x['type'] else 0, axis =1)
data

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,type,posts
0,1,and intj moments sportscenter not top t...
1,0,i m finding the lack of me in these posts ver...
2,1,good one course to which i say i ...
3,1,dear intp i enjoyed our conversation the o...
4,0,you re fired that s another silly misconce...
...,...,...
496,1,answer the subject of the thread you won ...
497,0,well when you figure out where to begin plea...
498,1,dude that sounds fantastic if you ever put a...
499,1,just picked this post to reply to but ive re...


In [98]:
from sklearn.model_selection import train_test_split

In [8]:
train_X,test_X,train_Y, test_Y = train_test_split(data.posts.values, data.type.values, random_state=0, test_size=0.2)

**using LSTM**

In [9]:
#importing packages 
import numpy as np
import pandas as pd
import nltk 
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
import string 
import re

In [10]:
#downloading stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [100]:
data

,type,posts
0,1,and intj moments sportscenter not top t...
1,0,i m finding the lack of me in these posts ver...
2,1,good one course to which i say i ...
3,1,dear intp i enjoyed our conversation the o...
4,0,you re fired that s another silly misconce...
...,...,...
496,1,answer the subject of the thread you won ...
497,0,well when you figure out where to begin plea...
498,1,dude that sounds fantastic if you ever put a...
499,1,just picked this post to reply to but ive re...


In [101]:
#process tweets
def process_tweet(tweet):
    stemmer=PorterStemmer()
    stopwords_english = stopwords.words('english') #stopwords english
    
    #removing all hashtags ,hyperlinks
    tweet = re.sub(r'\$\w*', '', tweet)
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    tweet = re.sub(r'#', '', tweet)
    
    tokenizer=TweetTokenizer(preserve_case=False, strip_handles=True,reduce_len=True)
    tweet_tokens=tokenizer.tokenize(tweet) #tokenize
    
    tweets_clean=[]
    
    #removing stopwords, removing punctuation and then stemming the word
    for word in tweet_tokens:
        if (word not in stopwords_english and word not in string.punctuation):   
            stem_word=stemmer.stem(word)
            tweets_clean.append(stem_word)
            
    return tweets_clean

In [102]:
#importing packages
import numpy as np
import pandas as pd
import  string 
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM


In [103]:
#creating vocablory 
vocab=[]
for tweet in df.posts:
  for word in process_tweet(tweet):
    if word not in vocab:
      vocab.append(word)

  
print("length of vocab:",len(vocab))

length of vocab: 17420


In [104]:
#process tweet
processed_tweet=[]
for t in df.posts:
  p_t=process_tweet(t)
  processed_tweet.append(p_t)


In [105]:

data['p_tweet']=processed_tweet
data.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,type,posts,p_tweet
0,1,and intj moments sportscenter not top t...,"[intj, moment, sportscent, top, ten, play, pra..."
1,0,i m finding the lack of me in these posts ver...,"[find, lack, post, alarm, sex, bore, posit, of..."
2,1,good one course to which i say i ...,"[good, one, cours, say, know, bless, curs, abs..."
3,1,dear intp i enjoyed our conversation the o...,"[dear, intp, enjoy, convers, day, esoter, gab,..."
4,0,you re fired that s another silly misconce...,"[fire, anoth, silli, misconcept, approach, log..."


In [106]:
label=[]
for i in data['type']:
  if i==1:
    label.append([0,1])
  elif i==0:
    label.append([1,0])

data['label']=label 
df.head()   

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,type,posts,p_tweet,label
0,1,and intj moments sportscenter not top t...,"[intj, moment, sportscent, top, ten, play, pra...","[0, 1]"
1,0,i m finding the lack of me in these posts ver...,"[find, lack, post, alarm, sex, bore, posit, of...","[1, 0]"
2,1,good one course to which i say i ...,"[good, one, cours, say, know, bless, curs, abs...","[0, 1]"
3,1,dear intp i enjoyed our conversation the o...,"[dear, intp, enjoy, convers, day, esoter, gab,...","[0, 1]"
4,0,you re fired that s another silly misconce...,"[fire, anoth, silli, misconcept, approach, log...","[1, 0]"


In [118]:
tokenizer = Tokenizer(num_words=17420, split=' ') 
tokenizer.fit_on_texts(data['p_tweet'].values)
X = tokenizer.texts_to_sequences(data['p_tweet'])
X=pad_sequences(X)
y=data['label'].to_list()
y=np.array(y)

In [119]:
embeding_dim=128
lstm_out=196
model = Sequential()
model.add(Embedding(input_dim=len(vocab),output_dim=embeding_dim,input_length = 895))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))


In [120]:
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])

In [121]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.15, random_state = 42)

In [122]:
model.fit(X_train, y_train,validation_data = (X_test,y_test),epochs = 3, batch_size=32)

Epoch 1/3
14/14 [==============================] - 82s 5s/step - loss: 0.5846 - accuracy: 0.7718 - val_loss: 0.4979 - val_accuracy: 0.8026
Epoch 2/3
14/14 [==============================] - 75s 5s/step - loss: 0.4801 - accuracy: 0.7788 - val_loss: 0.4843 - val_accuracy: 0.8026
Epoch 3/3
14/14 [==============================] - 75s 5s/step - loss: 0.3316 - accuracy: 0.8165 - val_loss: 0.5384 - val_accuracy: 0.7632


In [123]:
model.evaluate(X_test,y_test)

3/3 [==============================] - 2s 580ms/step - loss: 0.5384 - accuracy: 0.7632


[0.5383621454238892, 0.7631579041481018]

**saving the model**

In [124]:
model.save('tweet_introvert_extrovert_1.h5')

**loading the model**

In [125]:
from tensorflow import keras
model = keras.models.load_model('tweet_introvert_extrovert_1.h5')


## **Testing LSTM model**

In [128]:
def testing(tweet):
  processed_tweet=process_tweet(tweet)
  tweet_to_token=tokenizer.texts_to_sequences(processed_tweet)
  x1=[]
  for i in tweet_to_token:
    x1.append(i[0])
  padded_processed_tweet=pad_sequences([x1],maxlen=895,padding='pre')
  pred=model.predict(padded_processed_tweet)
  pred=np.argmax(pred)
  return pred


In [129]:
testing("it was super nice")

1

In [130]:
testing("   and intj moments     sportscenter not top ten plays     pranks   what has been the most life changing experience in your life           on repeat for most of today    may the perc experience immerse you    the last thing my infj friend posted on his facebook before committing suicide the next day  rest in peace      enfj7  sorry to hear of your distress  it s only natural for a relationship to not be perfection all the time in every moment of existence  try to figure the hard times as times of growth  as      84389  84390             welcome and stuff       game  set  match    prozac  wellbrutin  at least thirty minutes of moving your legs  and i don t mean moving them while sitting in your same desk chair   weed in moderation  maybe try edibles as a healthier alternative      basically come up with three items you ve determined that each type  or whichever types you want to do  would more than likely use  given each types  cognitive functions and whatnot  when left by      all things in moderation   sims is indeed a video game  and a good one at that  note  a good one at that is somewhat subjective in that i am not completely promoting the death of any given sim      dear enfp   what were your favorite video games growing up and what are your now  current favorite video games   cool      appears to be too late   sad    there s someone out there for everyone    wait    i thought confidence was a good thing    i just cherish the time of solitude b c i revel within my inner world more whereas most other time i d be workin    just enjoy the me time while you can  don t worry  people will always be around to      yo entp ladies    if you re into a complimentary personality well  hey        when your main social outlet is xbox live conversations and even then you verbally fatigue quickly       i really dig the part from 1 46 to 2 50     because this thread requires it of me    get high in backyard  roast and eat marshmellows in backyard while conversing over something intellectual  followed by massages and kisses      for too many b s in that sentence  how could you  think of the b    banned for watching movies in the corner with the dunces    banned because health class clearly taught you nothing about peer pressure    banned for a whole host of reasons      two baby deer on left and right munching on a beetle in the middle   2  using their own blood  two cavemen diary today s latest happenings on their designated cave diary wall   3  i see it as      a pokemon world  an infj society  everyone becomes an optimist   49142     all artists are artists because they draw  it s the idea that counts in forming something of your own    like a signature    welcome to the robot ranks  person who downed my self esteem cuz i m not an avid signature artist like herself   proud    banned for taking all the room under my bed  ya gotta learn to share with the roaches      for being too much of a thundering  grumbling kind of storm    yep    ahh    old high school music i haven t heard in ages      failed a public speaking class a few years ago and i ve sort of learned what i could do better were i to be in that position again  a big part of my failure was just overloading myself with too      i like this person s mentality  he s a confirmed intj by the way    to the denver area and start a new life for myself  ")

1

In [131]:
testing(" i m finding the lack of me in these posts very alarming    sex can be boring if it s in the same position often  for example me and my girlfriend are currently in an environment where we have to creatively use cowgirl and missionary  there isn t enough      giving new meaning to  game  theory    hello  entp grin   that s all it takes  than we converse and they do most of the flirting while i acknowledge their presence and return their words with smooth wordplay and more cheeky grins    this   lack of balance and hand eye coordination    real iq test i score 127  internet iq tests are funny  i score 140s or higher   now  like the former responses of this thread i will mention that i don t believe in the iq test  before you banish      you know you re an entp when you vanish from a site for a year and a half  return  and find people are still commenting on your posts and liking your ideas thoughts  you know you re an entp when you        over think things sometimes  i go by the old sherlock holmes quote   perhaps  when a man has special knowledge and special powers like my  own  it rather encourages him to seek a complex      cheshirewolf tumblr com  so is i  d   400 000   post   not really  i ve never thought of e i or j p as real functions   i judge myself on what i use  i use ne and ti as my dominates  fe for emotions and rarely si  i also use ni due to me strength      you know though  that was ingenious  after saying it i really want to try it and see what happens with me playing a first person shooter in the back while we drive around  i want to see the look on      out of all of them the rock paper one is the best  it makes me lol   you guys are lucky  d i m really high up on the tumblr system    so did you hear about that new first person shooter game  i ve been rocking the hell out of the soundtrack on my auto sound equipment that will shake the heavens  we managed to put a couple ps3 s in      no  the way he connected things was very ne  ne dominates are just as aware of their environments as se dominates   example  shawn spencer or patrick jane  both entps    well charlie i will be the first to admit i do get jealous like you do  i chalk it up to my 4w3 heart mixed with my dominate 7w8  7s and 8s both like to be noticed  4 s like to be known  not the same       d i ll upload the same clip with the mic away from my mouth  than you won t hear anything   ninja assassin style but with splatter    tik tok is a really great song  as long as you can mental block out the singer  i love the beat it makes me bounce    drop io v1swck0   d mic really close to my mouth and smokin aces  assassins ball playing in the background    sociable     extrovert  i m an extrovert and i m not sociable       sherlock in the movie was an entp  normally he s played as a extj  in the books he s an estj   as i said  the movie looked good except for it being called sherlock holmes      i never had fear of kissing a guy  i will kiss an animal too  so there was nothing to vanish  just personal taste and me not liking it   the guy i kissed didn t know me  it was one of those      sounds pretty much like my area and what i m going through right now trying to figure out which way i want to take my life  i want to do so many things  the biggest problem is that i know if i don t       d i was operating under the impression that you were female  i never looked at your boxy  okay  i help out my gay friends all the time and one of them has developed a little crush on me  i get red      t t you just described me  and i m living the worst nightmare  i m trapped in one place with one one around  only dull woods  if i was a serial killer this would be the perfect place but sadly i m      tbh  and biased  sounds like a shadowed infp  i think maybe he was hurt and turned estj  i can tell because he has some of the typical infp traits left over     checks list  i m sorry  it seems that you have came at a bad time  we ve already reached our quota of infjs  however  being you re female and i like females i will make you a deal  i will kick one      i m antp  leaning toward e   i m easy for both entps and intps to identify with       i also imagine entp s interrogations would go a little bit like jack s from 24 except more mechanical  rigging up shock treatment equipment in an abandoned building out of an old car batty  jumper      it was a compliment    trust me  i m just as psychopathic  d except i have emoticons  they re just weird ones  like laughing when i get hurt or at people running themselves over with their lawn mower          it s like a theme for where i live and that is why i know it by heart      i usual don t leave until the thing ends  but in the mean time  in between times  you work your thing  i ll work mine  d   d i m the mbp  pleasure to meet you    damn  need to trust my instincts more i would have been closer i was going to say infp    exfp  leaning toward s with the way she responded    d my friends  even my gay and lesbian ones  always come to me for advice    i bow to my entp masters entps are so great  if it wasn t for entps i wouldn t have been able to build what i m building  duck duck  duck  shotgun   what  me  i never do that           because its hard to be sad about losing someone you like when you knew you were right and give yourself a big pat on the back because you re awesome and always correct    oh  you don t have to tell me that most of them are stupid  i know this  that is why i play with them and it makes me laugh   d as i m going to take neuropsychology and i have a few psychologist       d i m a nightowl  i wake up between 6 7pm and stay awake till 10 11 30am    personal opinion backed by theory would suggest that intps are the most socially difficult  while intjs can be socially indifferent but they will also use social situations if the the need arises       personal stocks that i have on my desktop that i ve downloaded from random stock sites and stock photobuckets    i ll tell you when i open photoshop      glad you like it static     d thanks         made for a friend  several hours of work  i constructed every line by         static     i ll have to get to your avatar later if one of my fellow teammates doesn t    psychologist don t keep me around long enough to diagnosis me  i like to toy with them  what i have diagnosis myself with and had a few psychologist friends    a few other friends  tell me i have is    ")

0

In [132]:
testing(" you re fired    that s another silly misconception  that approaching is logically is going to be the key to unlocking whatever it is you think you are entitled to    nobody wants to be approached with bs      but guys    he really wants to go on a super duper long ass vacation  c mon guys  his boss just doesn t listen or get it  he even approached him logically and everything    never mind  just go on permanent vacation    two months  i wouldn t be crazy about the idea  if you are really his best employee  then that s what may be cooking him  who wants their most reliable asset gone for that long   entj employer      lol  its not like our views were unsolicited  what a victim    sometimes i just really like impoverished rap music      probably would have lost it too  his stonewalling was insignificant  he was being an idiot over semantics that didn t have any impact on anything  it s one thing to stone wall on critical shit  but      i would have just taken the project and gave them something later    the court deposition wasn t a skit  that actually happened  it was a verbatim reenactment    and her ass    i ll flirt a woman for a piece of candy at the grocery store  of course  i use whatever i have at my disposal to get what i want in the most efficient manner possible  if that means flirting that        in the comments she gave more info  then was shut down because she went from having a crush on her boss and wanted to know how to impress him  to her boss is married and she wasn t interested  i      what s this we thing about    callaendia are you the same infp girl who was on the entj facebook forum with this similar question  i just have to ask because the times are so close together  and the scenario is so similar    3x   11 25 i do not modify myself to suit others  however i do like the spotlight    i think the easiest and most efficient approach is a tarp  jigsaw  and mulcher  but that s just my personal preference  not all entjs are the same    just don t hump my leg      what are you like a chihuahua or some shit    i once dumped a girl in college who i was having a 6 month romp with over making me late for class  she said i was petty for it    dead xd     perfect  i don t like dealing with middle men either  you can talk directly to me        i can sell you the dream      lol dead  ha    the point is that when you are making a point of thinking critically but not thinking beyond common sense  you aren t using your brain  a little bit of effort now to have an understanding of the      that s just common sense  no  that s just a value judgement backed by popular opinion from social stupidity laziness  that s only doing what works instead of pushing harder and doing what is      when it comes to children  they either cry or climb all over me  there is never a gray area    i don t drink  but when i drank   beer  hardcore ciders and sweet brown ales   liquor  vodka chased with cran or water    my preferred drink  wine  anything dark  dry and sweet  there are so      well that escalated quickly       you deal with them    tl dr fuuuuu   those people who think they know everything are a great annoyance to those of us who do ia    darth alpha  i would never claim it to be healthy either  just in my best interest    on loyalty   i am loyal to the vision  i am loyal to those who share the vision  i am not into co dependency in terms of the vision cannot be accomplished without others involved  i stay on course       your answer was worth more time to entertain than the op    hey  msbossypants are you down for a debate on ayn rand vs marx  maybe we should talk about our poor fi  oh i know  let s try to correlate testing entj with being sociopathic    laughing    hitler was no exception  this concept has  is and always will be a principle  tenet  and rule  politics  ideology  religion  eating  shitting usual   and if it s not  then there is nothing to      hitler thought he was doing great things  stalin thought he was justified  these guys thought they were uncorrupt  they viewed any heretic to their dogmas as morally bankrupt  and so did many of      meh  there were plenty of evil people who did good things  and plenty of good people who got blood on their hands achieving and attaining higher goals  bad situations that had good long term effects       you would have to ask the op  he is asking ambiguous questions  i would go with whatever is more efficient effective vs what is perceived as good every time in the good vs evil dichotomy  it in my      good vs bad     good vs evil one is an objective positive vs negative dichotomy  the other is a subjective interpretation of characteristics    nameuser  so to be evil may be a better solution and      i think they are for sure idiots  but i also think these people went out of their way to fuck their lives up  did they expose themselves on the internet  yes  does this warrant or validate a reason      i really reeeeeeaaallly hate rigidly pc thumb sucking nanny bitches  they remind me of five year olds who are trying too hard to gain their parents approval by taddling  these are the same people       i think people need to get some skin  even if these people are wrong or racist  that s just a waste of time  energy  and resources  these people end up being the focus for being petty and overboard        not trying to be condescending when i tell you  all those things you have mentioned people saying and have written off as all i have been met with are legit arguments  they don t need to be      the brevity of my posts is because you don t take much to refute  just because you aren t acknowledging what people are saying doesn t mean they are saying nothing  ")

0

In [133]:
testing("i m not sure  that s a good question  the distinction between the two is so dependant on perception  to quote robb flynn    the hate you feel is nothing more  than love you feel to win this war      good question  it s tough to say for sure but i loved winona ryder as lydia in beetlejuice       been lonely for much of my time  for a while now i ve been working on changing   how to think    and one way is trying to find the positive in everything  no matter how bleak it might seem  i      i hope i can look back at this current stretch of time and think    thank god i m not there anymore and things are so much better    what an ass kickin  i m taking    help or a voice of reason  other than my own distorted  in my life  i m completely frusterated and feel like i m on the brink  i m carrying the weight of everything negative i ve ever had in my life      think i posted this before but i m still feelin  it         thoughts of acknowledging and getting lost in the depth love that we share for one another  it could be the most mundane thing but every now and the simplest act will make me appreciate the        been told that i m ocd and i m aware of it  for me it s more than just obsessive cleaning and things of that nature  i ve come to understand how it affects things such as obsessing on a thought        working on a song i ve been trying to learn for just about a year  was hoping to finish it before the new year but still have a ways to go    sons of anarchy is the first tv series i ve followed since the x files way back in the day      not a big book person but every now and then i go through stages and force myself to read more than i normally do  one book that always stands out in terms of interest while reading was   venus        got the best of me today  and has been a lot lately  having a couple heinekens right now and feel a bit better     this is a difficult time  and is going to be for a bit  gotta maintain    find balance  get more in harmony with life and position myself to capitalize on any opportunity that presents itself  within that lies the path to a more fulfilled life  at least i m hoping         not a touchy feely person and don t care for folks i don t know feeling up on me  my response depends greatly on the person doing it  some people are just naturally affectionate and it s their            very frusterated  wrestling with thought  and not winning    escape from alcatraz     grateful to still have a little time to make good of all the things i ve learned the hard way through countless mistakes and wrong turns      great song to start the day        hurt a little  other than that i feel the same old usual indifference   p   an early 90 s style ford mustang gt with the more square body  in all reality  one can be found for a few thousand dollars which makes this dream car very attainable         was kind of intended to have a dark  terrible to feel about it  depression has many depths  and those levels vary from person to person  i ve dealt with serious depression my whole life and it        food   pizza and chicken wings  cheesecake or pumpkin pie  comfort drink   redbull  guinness or red wine   still grinding out this song on guitar  it s coming along but slowly and most certainly not easily    some vague advice i can give is this  it can always be worse and in most cases it probably will be someday  so it works to look at any current situation like this  it helps to appreciate what you      content  not happy but more content than i ve been in a while  a long f n while  i ll take it    for breakfast i had a piece of grilled chicken and a guinness     seven  the decent  rocky  heat  silence of the lambs     flynn of machine head  michael amott of arch enemy  jerry cantrell of aic   edit  forgot to include zakk wylde  a musical genius    pucca   that shirt could be a lot of fun indeed           watching a video of megadeth live on the big four tour      it back with silk  great stuff      some down right now          currently desire for july 20th to roll around so i can finally see machine head live at this years mayhem festival  f  k yeah      ")

1

In [134]:
df=pd.read_csv('/content/tweets_new_data.csv')
df

,Tweets
0,President believed nothing could achieved with...
1,GhostStory Sometimes tries simply cannot paper...
2,Aut Starting online number slips issued sheet ...
3,servant someone above banks because guarantor ...
4,ThexExclusive Airdrop 00available participants...
...,...
1730,government Public Health England adopt rationa...
1731,Thousands gathered today showered roses countr...
1732,DEFAMING works earns himself SELFLESSLY COMMON...
1733,Unfortunately common problem People therapists...


In [135]:
tweets=df['Tweets'].to_list()

In [136]:
tw_l=[]
for i in tweets:
  pt=process_tweet(i)
  new_t=[]
  for j in pt:
    if j in vocab:
      new_t.append(j)
  tw_l.append(new_t)
tw_l


[['presid',
  'believ',
  'noth',
  'could',
  'achiev',
  'without',
  'improv',
  'corrupt',
  'declar',
  'corrupt',
  'biggest',
  'fight',
  'poverti',
  'launch',
  'comprehens',
  'campaign',
  'corrupt',
  'level'],
 ['sometim',
  'tri',
  'simpli',
  'cannot',
  'paper',
  'rather',
  'thumb',
  'noth',
  'hammer',
  'regardless',
  'despit',
  'opposit',
  'failur'],
 ['start',
  'onlin',
  'number',
  'slip',
  'issu',
  'sheet',
  'upload',
  'websit',
  'student',
  'check',
  'portal',
  'updat',
  'instruct',
  'avail',
  'websit'],
 ['servant',
  'someon',
  'bank',
  'bank',
  'accept',
  'daughter',
  'profession',
  'centuri',
  'fight',
  'equal',
  'right',
  'women'],
 ['particip', 'copyright', 'click', 'particip'],
 ['launch',
  'give',
  'member',
  'chanc',
  'uniti',
  'follow',
  'thing',
  'twitter',
  'telegram',
  'tweet',
  'favourit',
  'influenc'],
 ['uniti',
  'protocol',
  'nativ',
  'begin',
  'today',
  'criteria',
  'member',
  'discord',
  'group'

In [137]:
df['pro_tweet']=tw_l

In [138]:
df

,Tweets,pro_tweet
0,President believed nothing could achieved with...,"[presid, believ, noth, could, achiev, without,..."
1,GhostStory Sometimes tries simply cannot paper...,"[sometim, tri, simpli, cannot, paper, rather, ..."
2,Aut Starting online number slips issued sheet ...,"[start, onlin, number, slip, issu, sheet, uplo..."
3,servant someone above banks because guarantor ...,"[servant, someon, bank, bank, accept, daughter..."
4,ThexExclusive Airdrop 00available participants...,"[particip, copyright, click, particip]"
...,...,...
1730,government Public Health England adopt rationa...,"[govern, public, health, england, adopt, ratio..."
1731,Thousands gathered today showered roses countr...,"[thousand, gather, today, shower, rose, countr..."
1732,DEFAMING works earns himself SELFLESSLY COMMON...,"[defam, work, earn, common, peopl, everyth, po..."
1733,Unfortunately common problem People therapists...,"[unfortun, common, problem, peopl, therapist, ..."


In [139]:
def testing(tweet):
  #processed_tweet=process_tweet(tweet)
  tweet_to_token=tokenizer.texts_to_sequences(tweet)
  x1=[]
  for i in tweet_to_token:
    x1.append(i[0])
  padded_processed_tweet=pad_sequences([x1],maxlen=895,padding='pre')
  pred=model.predict(padded_processed_tweet)
  pred=np.argmax(pred)
  return pred


In [ ]:
result=[]
for i in df['pro_tweet'].to_list():
  result.append(testing(i))

In [141]:
df['result']=result
df=df[['Tweets','result']]
df.head()

,Tweets,result
0,President believed nothing could achieved with...,1
1,GhostStory Sometimes tries simply cannot paper...,1
2,Aut Starting online number slips issued sheet ...,1
3,servant someone above banks because guarantor ...,1
4,ThexExclusive Airdrop 00available participants...,1


In [143]:
df.to_csv('df.csv')